In [1]:
print('hello world')

hello world


In [2]:
print('second commit test')

second commit test


In [3]:
import zipfile
import os

# Define the path to the uploaded zip file
zip_path = "AQI_BY_COUNTY_25_YEARS.zip"
extractedDir = "/aqi_data"

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extractedDir)

# List extracted files
extracted_files = os.listdir(extractedDir)
extracted_files

['AQI_BY_COUNTY_25_YEARS']

In [5]:
subdir_path = os.path.join(extractedDir, 'AQI_BY_COUNTY_25_YEARS')
subdir_contents = os.listdir(subdir_path)
#subdir_contents

In [6]:
import pandas as pd

# Load and combine all CSV files into a single DataFrame
all_files = [os.path.join(subdir_path, f) for f in subdir_contents if f.endswith('.csv')]
df_list = [pd.read_csv(file) for file in all_files]
aqi_df = pd.concat(df_list, ignore_index=True)

# Show the first few rows and column names for inspection
#aqi_df.head(), aqi_df.columns.tolist()

In [7]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter nbextension install --py widgetsnbextension --sys-prefix

In [8]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load the preprocessed data (assumes it's loaded into `aqi_df`)
aqi_df = pd.read_csv("AQI_BY_COUNTY_25_YEARS/annual_aqi_by_county_2024.csv")
for year in range(2000, 2024):
    yearly = pd.read_csv(f"AQI_BY_COUNTY_25_YEARS/annual_aqi_by_county_{year}.csv")
    aqi_df = pd.concat([aqi_df, yearly], ignore_index=True)

# Widgets for interaction
state_widget = widgets.Dropdown(
    options=sorted(aqi_df['State'].unique()),
    description='State:',
    layout=widgets.Layout(width='50%')
)

county_widget = widgets.Dropdown(
    options=[],
    description='County:',
    layout=widgets.Layout(width='50%')
)

year_range = widgets.IntRangeSlider(
    value=[2000, 2024],
    min=2000,
    max=2024,
    step=1,
    description='Year Range:',
    continuous_update=False,
    layout=widgets.Layout(width='90%')
)

output = widgets.Output()

def update_county_options(*args):
    selected_state = state_widget.value
    counties = sorted(aqi_df[aqi_df['State'] == selected_state]['County'].unique())
    county_widget.options = counties

state_widget.observe(update_county_options, 'value')
update_county_options()

def show_summary(change=None):
    with output:
        clear_output()
        state = state_widget.value
        county = county_widget.value
        year_min, year_max = year_range.value

        df_filtered = aqi_df[
            (aqi_df['State'] == state) &
            (aqi_df['County'] == county) &
            (aqi_df['Year'] >= year_min) &
            (aqi_df['Year'] <= year_max)
        ]

        if df_filtered.empty:
            print("No data available for the selected options.")
        else:
            display(df_filtered.describe(include='all'))
            display(df_filtered.groupby('Year')[[
                'Good Days', 'Moderate Days', 'Unhealthy for Sensitive Groups Days',
                'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days',
                'Max AQI', 'Median AQI', 'Days with AQI']].sum())

submit_btn = widgets.Button(description="Show Report", button_style='primary')
submit_btn.on_click(show_summary)

# Display all widgets and output
ui = widgets.VBox([state_widget, county_widget, year_range, submit_btn, output])
display(ui)
